# Función lector de mascarillas

En base a la observacion de etiquetas de mascarillas y su lectura en tesseract, montar un Count Vectorizer que recoja las palabras o n-gramas clave que suelen indicar si: 

* La mascarilla es reutilizable o no
    * En caso que sea reutilizable, el número de lavados recomendado
    * En caso que no sea reutilizable, el número de horas máximas 
* El certificado de la mascarilla (UNE 0064...
* Su capacidad de filtración


In [1]:
# cargo librerias
from PIL import Image
import pytesseract as tess
import cv2
from skimage.io import imread
import os
import matplotlib.pyplot as plt
import re
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd


In [12]:
# creo la expresion que vectorizara
palabras_clave = ['no reutilizable 4 horas 8 horas lavar 25 veces']

vectorizer = CountVectorizer(binary = True, ngram_range = (1, 2), token_pattern = '(?u)\\b\\w+\\b')
vectorizer.fit(palabras_clave)

vectorizer.vocabulary_

{'no': 11,
 'reutilizable': 13,
 '4': 2,
 'horas': 6,
 '8': 4,
 'lavar': 9,
 '25': 0,
 'veces': 15,
 'no reutilizable': 12,
 'reutilizable 4': 14,
 '4 horas': 3,
 'horas 8': 7,
 '8 horas': 5,
 'horas lavar': 8,
 'lavar 25': 10,
 '25 veces': 1}

In [13]:
# funcion

def lector(vectorizador, direccion_image):
    
    # diccionario con el output 
    resultados = {"Reut o no" : None, 
                 "Uso" : None,
                 "Certificados" : None}
    # cargo imagen

    imagen = Image.open(direccion_image)
    
    # la convierto en texto

    texto = tess.image_to_string(imagen)
    
    # limpio el texto

    texto = ' '.join(texto.split())

    # meto el texto en una lista (requerimiento del vectorizador)
    lista = []
    lista.append(texto)
    
    # la encajo contra el vectorizador

    lista_vectorizada = vectorizador.transform(lista)
    
    # resultados
    
    if lista_vectorizada.toarray()[:, vectorizador.vocabulary_['no reutilizable']][0] == 1:
        resultados["Reut o no"] = "No reutilizable"
        if lista_vectorizada.toarray()[:, vectorizador.vocabulary_['4 horas']][0] == 1:
            resultados["Uso"] = "4 horas"
        elif lista_vectorizada.toarray()[:, vectorizador.vocabulary_['8 horas']][0] == 1:
            resultados["Uso"] = "8 horas"
    
    elif lista_vectorizada.toarray()[:, vectorizador.vocabulary_['no reutilizable']][0] == 0:
        resultados["Reut o no"] = "Reutilizable"
        if lista_vectorizada.toarray()[:, vectorizador.vocabulary_['lavar 25']][0] == 1:
            resultados["Uso"] = "25 lavados"     
    return resultados
        
        

In [14]:
# ejemplo con una mascarilla REUTILIZABLE
lector(vectorizer, direccion_image = 'Etiquetas/MASK reut.png')

{'Reut o no': 'Reutilizable', 'Uso': '25 lavados', 'Certificados': None}

In [15]:
# ejemplo con una mascarilla NO REUTILIZABLE
lector(vectorizer, direccion_image = 'Etiquetas/Mercadona.png')

{'Reut o no': 'No reutilizable', 'Uso': '4 horas', 'Certificados': None}